# Results Validation
This notebook loads the reconstructed data and compares the results with the available data.

## 0 - Setup

### 0.1 - Imports
Load the necessary dependencies.

In [1]:
!pip install ipywidgets 

In [2]:
from ydata.connectors import LocalConnector
from ipywidgets import interact

/opt/conda/lib/python3.7/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


## 0.2 - Auxiliary Functions
The auxiliary functions are custom-designed utilities developed for the REE use case.

In [3]:
from utils import setting_index_data
from imputation import resample_station_data
from results import meter_statistics
from results_viz import plot_timeseries, compare_acf, plot_multicolor_line, plot_hist, compare_lags, plot_metric_byperiod, polar_graph

## 1 - Load Data

In [4]:
connector = LocalConnector()

In [5]:
# Load the reconstructed data for the given device
reconstructed = setting_index_data(connector.read_file('whole_year_reconstructed.csv'))

In [6]:
# Load the original data for the given device
original = setting_index_data(connector.read_file('train_allmeters.csv'))

## 2 - Results per Meter

In [7]:
@interact(meter=set(reconstructed['station']))
def plot_meter_results(meter):
    meter_reconstructed = reconstructed[reconstructed['station']==meter]
    meter_original = original[original['station']==meter]
    
    # Convert the original data into resampled for full year 
    original_year = resample_station_data(meter_original)

    meter_statistics(original_year)
    plot_timeseries(data=original_year, feature='speed', title='Original Data for '+meter)
    plot_multicolor_line(original_year, meter_reconstructed, title='Reconstruction of '+meter)
    compare_acf(original_year, meter_reconstructed, title='Hourly Autocorrelation')
    plot_hist(original_year, meter_reconstructed, title='Distribution Overlap - Original and Reconstructed Series - '+meter)
    compare_lags(original_year, meter_reconstructed, 'speed')
    plot_metric_byperiod(original_year, reconstructed, 'speed')
    polar_graph(original_year, reconstructed)

interactive(children=(Dropdown(description='meter', options=('asosabambasi', 'seladingay', 'sure', 'ziway', 'k…